In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from sparsify.data import chunk_and_tokenize
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

/home/adam/.conda/envs/jax311/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:

tokenizer = AutoTokenizer.from_pretrained("EleutherAI/pythia-31m")
tokenizer.pad_token_id = 1
tokenizer.eos_token_id = 0

# 'en' subset, 'test' split
data = load_dataset("EleutherAI/lambada_openai", name="en", split="test", trust_remote_code=True)
tokens = chunk_and_tokenize(data, tokenizer, max_seq_len=256, text_key="text")["input_ids"]

In [4]:
from tqdm import tqdm

In [10]:
tokens.device, tokens.shape

(device(type='cpu'), torch.Size([1635, 256]))

In [11]:
%%time
for i in tqdm(range(len(tokens))):
    foo = tokens[i].cuda()
del foo

100%|██████████| 1635/1635 [00:00<00:00, 48549.65it/s]

torch.Size([256])
torch.Size([256])
torch.Size([256])
torch.Size([256])
torch.Size([256])
torch.Size([256])
torch.Size([256])
torch.Size([256])
torch.Size([256])
torch.Size([256])
torch.Size([256])
torch.Size([256])
torch.Size([256])
torch.Size([256])
torch.Size([256])
torch.Size([256])
torch.Size([256])
torch.Size([256])
torch.Size([256])
torch.Size([256])
torch.Size([256])
torch.Size([256])
torch.Size([256])
torch.Size([256])
torch.Size([256])
torch.Size([256])
torch.Size([256])
torch.Size([256])
torch.Size([256])
torch.Size([256])
torch.Size([256])
torch.Size([256])
torch.Size([256])
torch.Size([256])
torch.Size([256])
torch.Size([256])
torch.Size([256])
torch.Size([256])
torch.Size([256])
torch.Size([256])
torch.Size([256])
torch.Size([256])
torch.Size([256])
torch.Size([256])
torch.Size([256])
torch.Size([256])
torch.Size([256])
torch.Size([256])
torch.Size([256])
torch.Size([256])
torch.Size([256])
torch.Size([256])
torch.Size([256])
torch.Size([256])
torch.Size([256])
torch.Size

In [12]:
model = AutoModelForCausalLM.from_pretrained("EleutherAI/pythia-14m").cuda()
model.eval()

input_ids = tokens[:10].to(model.device)
logits = model(input_ids).logits



The `GPTNeoXSdpaAttention` class is deprecated in favor of simply modifying the `config._attn_implementation`attribute of the `GPTNeoXAttention` class! It will be removed in v4.48


In [ ]:
logits_cpu = 

In [20]:
%%time

logits_cpu = []

batch_size = 100
for i in tqdm(range(0, len(tokens), batch_size)):
    input_ids = tokens[i:i+batch_size].cuda()
    logits = model(input_ids).logits
    logits_cpu.append(logits.cpu())

logits_cpu = torch.cat(logits_cpu, dim=0)

  0%|          | 0/17 [00:00<?, ?it/s]

100%|██████████| 17/17 [00:31<00:00,  1.87s/it]


CPU times: user 31.8 s, sys: 3min 37s, total: 4min 9s
Wall time: 40.5 s


In [21]:
%%time

batch_size = 100
for i in tqdm(range(0, len(tokens), batch_size)):
    logits_gpu = logits_cpu[i:i+batch_size].cuda()

100%|██████████| 17/17 [00:11<00:00,  1.53it/s]

CPU times: user 8.98 s, sys: 2.15 s, total: 11.1 s
Wall time: 11.1 s


In [16]:
logits_cpu = logits_cpu.squeeze(1)

In [17]:
logits_cpu.shape, logits_cpu.device

(torch.Size([1635, 256, 50304]), device(type='cpu'))

In [ ]:
1635 * 256 / 2048

In [32]:
logits.shape, logits.device

(torch.Size([10, 256, 50304]), device(type='cuda', index=0))

In [18]:
# memory usage
logits_cpu.storage().size() * 4 / 1024**2




/tmp/ipykernel_3945469/815441628.py:2: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  logits_cpu.storage().size() * 4 / 1024**2


80319.375